In [38]:
import scipy as sp
from scipy.signal import lfilter
import numpy as np
import soundfile as sf
import librosa
import matplotlib.pyplot as plt

In [101]:
def deltas(x, w=9):
    hlen = int(np.floor(w/2))
    w = 2*hlen + 1
    win = np.arange(hlen, -hlen - 1, -1)
    xx = np.tile(x[:, 0], (1, hlen))
    
    #xx = np.concatenate([np.tile(x[:, 0], (1, hlen)), x, np.tile(x[:, -1], (1, hlen))], axis=1)
    print(x.shape)


def triangular_membership_function(x, a, b, m):
    result = np.zeros((x.shape[0]))
    for i in np.arange(x.shape[0]):
        if x[i] <= a:
            result[i] = 0
        elif x[i] > a and x[i] <= m:
            result[i] = (x[i] - a) / (m - a)
        elif x[i] > m and x[i] < b:
            result[i] = (b - x[i]) / (b - m)
        else:
            result[i] = 0
    return result
            
    


def compute_lfcc(audio, fs, window_length=20, nfft=512, n_filter=20):
    audio = lfilter([1, -0.97], 1, audio)
    
    
    X = np.abs(librosa.stft(
        audio, 
        window = 'hamming',
        n_fft = nfft,
        win_length = window_length,
        hop_length = window_length // 2)
    )*2

    f = fs/2 * np.linspace(0, 1, nfft//2 + 1)
    fil_bandwidth_sf = np.linspace(np.min(f), np.max(f), n_filter + 2)
    
    filterbank = np.zeros((nfft//2 + 1, n_filter))
    
    for i in range(n_filter):
        filterbank[:, i] = triangular_membership_function(f, fil_bandwidth_sf[i], 
                                                          fil_bandwidth_sf[i+1], fil_bandwidth_sf[i+2])
    
    filbanksum = np.dot(X.T, filterbank)

    log_filbanksum = np.log10(filbanksum + 1e-16)
    lfcc = sp.fftpack.dct(log_filbanksum, axis=0, norm='ortho')[1:n_filter]
    
    stat = lfcc.T
    

In [99]:
audio, fs = sf.read('audio/bonafide/LA_T_1011221.flac')
lfcc = compute_lfcc(audio, fs)

In [102]:
deltas(np.identity(10))

(10, 10)


In [90]:
np.identity(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [82]:
a= np.arange(5,-6, -1)

In [83]:
a[-1]

-5